In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import suppression as s
import utils
import linear_model as model

In [ ]:
gaba_fn = 'gaba_data.txt'
supp_fn = 'supp_data_individual_20170427.txt'

In [ ]:
sdf = utils.load_psychophys(supp_fn)
gdf = utils.load_gaba(gaba_fn)

In [ ]:
plot_dir = "plots/redo-201807"

## Analyze tasks separately (before subsetting to include common subjects)##

In [ ]:
pp_subjs = np.unique(sdf.Subject)
gaba_subjs = np.unique(gdf.subjName)
print(pp_subjs, len(pp_subjs))
print(gaba_subjs, len(gaba_subjs))

### GABA only analyses

### GABA t-test, CON v AMB

In [ ]:
pop_group = gdf.groupby("Population")
pop_group.describe()

In [ ]:
gaba_per_group = [col for col_name, col in pop_group['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group)
print(tstat, pval)

### GABA violin plot, n=31 (all)

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_diffs_n31.pdf") as pdf:
    fig = plt.figure(figsize=(6,8))  # create a figure object
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.violinplot(y='GABA',x='Presentation',hue='Population',data=gdf,split=True,inner='stick',ax=ax)
    ax.xaxis.set_visible(False)
    ax.set_ylabel("GABA (relative to Creatine)")
    plt.show(ax.figure)
    pdf.savefig(ax.figure)
    plt.close(ax.figure)
    plt.close('all')

## Select one psychophysical task's data ##

In [ ]:
task = 'SS'
sdf = sdf[sdf['Task']==task]

### Begin grouping data into conditions to model Subject's ThreshElev as a function of logRelContrast #

In [ ]:
pp_gvars = ['Task','Orientation','Presentation','Population','Subject','Eye','Trace'] # One condition
pp_gvars_base = pp_gvars + ['BaselineThresh']

In [ ]:
# create groups based on these grouping variables
groups = sdf.groupby(pp_gvars)
print(len(groups))
#for gv, g in groups:
#    print(gv)

## Find the RelMaskContrast at which NDE and DE most different within population using t-tests

### New way, moving the old way to functions in utils.py, 7-12-18

In [ ]:
gvars_test = ['Task','Orientation','Presentation','Population']
df_to_model = utils.find_xvalue_to_predict(sdf, gvars_test, test_func=st.ttest_ind)
df_to_model.head()

In [ ]:
n_pp_subs_thistask = len(np.unique(df_to_model.Subject))
print(n_pp_subs_thistask, len(np.unique((df_to_model[df_to_model['Population']=='Amblyope'])['Subject'])))

### Old way

In [ ]:
tt_df = sdf

In [ ]:
tt_df.head()

In [ ]:
gvars_ttest = ['Task','Orientation','Presentation','Population']
gvars_pair = gvars_ttest + ['BinNumber']

tt_grouped = tt_df.groupby(gvars_ttest)
print(len(tt_grouped))

for gv, g in tt_grouped:
    print(gv)

In [ ]:
tt_ind_binpred = tt_grouped.apply(utils.test_all_bins, gvars_pair, st.ttest_ind).reset_index()

In [ ]:
tt_ind_binpred.head()

In [ ]:
tt_welch_binpred = tt_grouped.apply(utils.test_all_bins, gvars_pair, st.ttest_ind, equal_var=False).reset_index()

### Add bin to predict to df that will be modeled

In [ ]:
sdf = pd.merge(sdf, tt_ind_binpred, on=gvars_ttest)
print(gvars_ttest)

In [ ]:
sdf.head()

In [ ]:
# make sure all conditions have the same bin number to predict within them
condition_groups = sdf.groupby(gvars_ttest + ['Eye'])
assert(np.all(condition_groups.apply(
        lambda g: np.all(g.BinNumberToPred==g.BinNumberToPred.iat[0])
    ).reset_index()))

In [ ]:
df_to_model_old = condition_groups.apply(utils.add_pred_col)

In [ ]:
df_to_model_old.head()

In [ ]:
n_pp_subs_thistask = len(np.unique(df_to_model.Subject))
print(n_pp_subs_thistask, len(np.unique((df_to_model[df_to_model['Population']=='Amblyope'])['Subject'])))

In [ ]:
assert(np.all(df_to_model == df_to_model_old))

### Linear model (defined in linear_model.py, uses lmfit)###

In [ ]:
# initialize parameters
model_params = model.parameters()

### Model all data points using linear model, generically named ###

In [ ]:
groups = df_to_model.groupby(pp_gvars) 

In [ ]:
preds = groups.apply(utils.model_threshold, model.err, model.thresh, model_params, ret='preds')

## Descriptive statistics

In [ ]:
preds.head(n=2)

In [ ]:
preds.groupby(["Orientation", "Presentation", "Population", "Eye"])["ThreshPred"].describe()

In [ ]:
s.group_facet_plots(preds, s.fit_plot,
                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subs_thistask}_TOP.pdf",
                    ['Task','Orientation','Presentation'], #each combo of this gets its own page
                    row='Population',col='Eye',# facet rows and columns
                    x="RelMaskContrast", y="ThreshElev", # x, y
                    hue="Subject",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshPred") 

In [ ]:
s.group_facet_plots(preds, s.fit_plot,
                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subs_thistask}_TO.pdf",
                    ['Task','Orientation'], #each combo of this gets its own page
                    row='Presentation',col='Eye',# facet rows and columns
                    x="RelMaskContrast", y="ThreshElev", # x, y
                    hue="Population",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshPred") 

####TODO: better plots, include the BinCenterRelMaskContrasts (maybe?) and the RelMCToPred

### Model the data again, but this time return parameters, not predictions ###

In [ ]:
print(pp_gvars_base)
groups_with_baseline = df_to_model.groupby(pp_gvars_base)

In [ ]:
# two-stage model
pfit = groups_with_baseline.apply(s.model_condition, model.two_stage_response_err, model.two_stage_thresh,
                                  twostage_params, ret='weights',
                                  supp_only=False).reset_index()

In [ ]:
# linear model
# pfit = groups_with_baseline.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh,
#                                   lm_params, ret='weights', predtype='linear', 
#                                   supp_only=False).reset_index()

In [ ]:
pfit.head()

In [ ]:
# melt the result of the modeling into long format for plotting
pfit_all_ppsub = pd.melt(pfit, id_vars=pp_gvars, var_name='measure')
pfit_all_ppsub.head()

In [ ]:
len(np.unique(pfit_all_ppsub.Subject))

In [ ]:
pfit_all_ppsub[pfit_all_ppsub.measure=="w_d"].describe()

# Test next part carefully, copy/paste

## subset to include common subjects

In [ ]:
use_subjs = list(np.intersect1d(pp_subjs, gaba_subjs))
print(use_subjs, len(use_subjs))

In [ ]:
sdf = sdf[sdf.Subject.isin(use_subjs)] # only subjects who did _the current_ pp task and GABA
len(np.unique(sdf.Subject))
# make sure there are only as many unique subjNames in sdf as there are entries in gdf.subjName
#assert(len(np.unique(sdf.Subject))==len(gdf.subjName))

In [ ]:
len(gdf[gdf['Population']=='Amblyope'])

In [ ]:
print(len(gdf), len(gdf[gdf.Population=='Amblyope']))

In [ ]:
gdf = gdf[gdf.subjName.isin(sdf.Subject)] # only subjects who did both tasks
print(len(gdf), len(gdf[gdf.Population=='Amblyope']))
n_this_task = len(gdf)

## Combine Psychophysics and GABA below

In [ ]:
#Grab the GABA measure for each subject and it to each observation for easy plotting
comb = pfit_all_ppsub.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])
print(len(comb))

#subset to include only two measures of interest: normalized thresh elev, and unnormed
comb_all_ppsub = comb[(comb['measure']=='ThreshPredCritical') | (comb['measure'] == 'ThreshPredCriticalUnnorm') | (comb['measure'] == 'BaselineThresh')]
print(len(comb_all_ppsub))

#subset to include only those subjects with GABA data
comb_gabappsub = comb_all_ppsub[~np.isnan(comb_all_ppsub['GABA'])]
print(len(comb_gabappsub), len(np.unique(comb_gabappsub.Subject)))

In [ ]:
#graphs!
with s.PdfPages('plots/collab-201710/gaba_vs_{}_n{}.pdf'.format(task, n_this_task)) as pdf:
    plot_groups = comb_gabappsub.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv, np.all(np.isnan(gr['value'])))
        #print(gr.columns)
        #if gv[0]=='SS' and gv[1]=='Cross': continue
        g2 = s.gaba_vs_psychophys_plot_2line(gv, gr)
        pdf.savefig(g2.fig)
        #g4 = s.gaba_vs_psychophys_plot_4line(gv, gr)
        #pdf.savefig(g4.fig)
        
    plt.close('all')

#### combine measures across the two eyes

In [ ]:
paired_obs = comb_gabappsub.groupby(['Task', 'Orientation', 'Population', 'Presentation', 'Subject', 'measure'])

def get_eyediff_value(g):
    if len(g)==2:
        value_diff = g[g['Eye']=='Nde'].value.iat[0] - g[g['Eye']=='De'].value.iat[0]
        #print(g.name, value_diff)
        return pd.Series([value_diff], ['Nde-De'])

In [ ]:
obs_diff = paired_obs.apply(get_eyediff_value).reset_index()

In [ ]:
obs_diff.head(n=6)

In [ ]:
comb_botheyes = obs_diff.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [ ]:
comb_botheyes.head()

In [ ]:
to_write = comb_botheyes[(comb_botheyes.Presentation == 'nDicho') & (comb_botheyes.measure == 'ThreshPredCriticalUnnorm')]

In [ ]:
to_write.to_csv('gaba_vs_ss_dicho.csv')

In [ ]:
print(len(np.unique(comb_botheyes.Subject)))

In [ ]:
#graphs!
with s.PdfPages('plots/collab-201710/gaba_vs_{}_combeyes_n{}.pdf'.format(task, n_this_task)) as pdf:
    plot_groups = comb_botheyes.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv)
        g2 = s.gaba_vs_psychophys_plot_2line_2eye(gv, gr)
        pdf.savefig(g2.fig)
        
    plt.close('all')

In [ ]:
# more graphs for presentation!
with s.PdfPages('plots/collab-201710/gaba_vs_{}_combeyes_n{}_condensed.pdf'.format(task, n_this_task)) as pdf:
    plot_groups = comb_botheyes.groupby(['Task','measure'])
    for gv, gr in plot_groups:
        print(gv)
        if "BaselineThresh" not in gv:
            g2 = s.gaba_vs_psychophys_plot_2line_2eye(gv, gr, row="Orientation", size=8, aspect=1.2)
            pdf.savefig(g2.fig)
        
    plt.close('all')